In [1]:
import pandas as pd
import collections
from functools import reduce

import sqlalchemy as salc
import json
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches


with open("/Users/pvuda/Development/il_ai_creds.json") as config_file:
    config = json.load(config_file)
    
from sqlalchemy import create_engine, text

database_connection_string = 'postgresql+psycopg2://{}:{}@{}:{}/{}'.format(config['user'], config['password'], config['host'], config['port'], config['database'])

connection_string = database_connection_string
engine = create_engine(connection_string)

In [2]:
def issue_msg_query(repo_org, repo_name):
    """
    Execute a SQL query to fetch issue message data for a given repository.

    Args:
    -----
        repo_org (str): The organization name of the repository.
        repo_name (str): The name of the repository.

    Returns:
    --------
        pd.DataFrame: Data frame containing issue message data with issue IDs, timestamps, and 
                      contributor IDs associated with each issue.
    """
    ism_query = salc.sql.text(f"""
                 SET SCHEMA 'augur_data';
                 SELECT
                    i.issue_id,
                    m.cntrb_id,
                    i.closed_at as timestamp
                FROM
                    repo_groups rg,
                    repo r,
                    issues i,
                    issue_message_ref imr,
                    message m
                WHERE
                    rg.repo_group_id = r.repo_group_id AND
                    i.repo_id = r.repo_id AND
                    i.issue_id = imr.issue_id AND
                    m.msg_id = imr.msg_id AND
                    r.repo_git = \'{f"https://github.com/{repo_org}/{repo_name}"}\'
                ORDER BY
                      timestamp DESC
        """)

    ism_data = pd.read_sql(ism_query, con=engine)
        
    # Convert the 'timestamp' column to datetime, this will help in identifying and handling missing values.
    ism_data['timestamp'] = pd.to_datetime(ism_data['timestamp'], errors='coerce')

    # Drop rows where 'timestamp' is NaT (missing)
    ism_data = ism_data.dropna(subset=['timestamp'])

    # reformat issue message data, combine contributor ids for each issue
    ism_data = ism_data.groupby('issue_id').agg({'cntrb_id': list, 'timestamp': 'last'}).reset_index()

    # remove issues with only one contributor (no connection to be made)
    ism_data = ism_data[ism_data['cntrb_id'].apply(lambda x: len(x) > 1)]
    # ism_data = ism_data.sort_values('timestamp', ascending=False)

    return ism_data

In [3]:
def commit_query(repo_org, repo_name):
    """
    Execute a SQL query to fetch commit data for a given repository.

    Args:
    -----
        repo_org (str): The organization name of the repository.
        repo_name (str): The name of the repository.

    Returns:
    --------
        pd.DataFrame: Data frame containing commit data with commit hash, timestamps, and corresponding 
                      author and committer IDs.
    """
    cmt_query = salc.sql.text(f"""
                    SET SCHEMA 'augur_data';
                    SELECT
                        DISTINCT c.cmt_commit_hash,
                        c.cmt_committer_timestamp as timestamp,
                        (SELECT ca.cntrb_id FROM contributors_aliases ca WHERE c.cmt_author_email = ca.alias_email) as author_id,
                        (SELECT ca.cntrb_id FROM contributors_aliases ca WHERE c.cmt_committer_email = ca.alias_email) as committer_id
                    FROM
                        repo_groups rg,
                        repo r,
                        commits c
                    WHERE
                        c.repo_id = r.repo_id AND
                        rg.repo_group_id = r.repo_group_id AND
                        r.repo_git = \'{f"https://github.com/{repo_org}/{repo_name}"}\' AND
                        c.cmt_author_email != c.cmt_committer_email
                    ORDER BY
                        timestamp DESC
            """)

    cmt_data = pd.read_sql(cmt_query, con=engine)
    cmt_data = cmt_data.dropna()
    # Convert the timestamp column to offset-naive datetime objects
    cmt_data['timestamp'] = cmt_data['timestamp'].apply(lambda x: x.replace(tzinfo=None) if x.tzinfo else x)

    return cmt_data

In [4]:
def pr_query(repo_org, repo_name): 
    """
    Execute a SQL query to fetch pull request data for a given repository.
    
    Args:
    -----
        repo_org (str): The organization name of the repository.
        repo_name (str): The name of the repository.

    Returns:
    --------
        pd.DataFrame: Data frame containing pull request data with pull request IDs, timestamps, 
                      contributor IDs, and reviewer IDs for each pull request.
    """
    pr_query = salc.sql.text(f"""
                  SET SCHEMA 'augur_data';
                  SELECT
                      pr.pull_request_id,
                      pre.cntrb_id,
                      prr.cntrb_id as reviewer,
                      pr.pr_created_at as timestamp
                  FROM
                      repo_groups rg,
                      repo r,
                      pull_requests pr,
                      pull_request_events pre,
                      pull_request_reviewers prr
                  WHERE
                      rg.repo_group_id = r.repo_group_id AND
                      pr.repo_id = r.repo_id AND
                      pr.pull_request_id = pre.pull_request_id AND
                      pr.pull_request_id = prr.pull_request_id AND
                      pre.cntrb_id != prr.cntrb_id AND
                      r.repo_git = \'{f"https://github.com/{repo_org}/{repo_name}"}\'
                  ORDER BY
                      timestamp DESC
          """)

    pr_data = pd.read_sql(pr_query, con=engine)
    pr_data = pr_data.dropna()

    return pr_data

In [5]:
def pr_msg_query(repo_org, repo_name): 
    """
    Execute a SQL query to fetch pull request message data for a given repository.

    Args:
    -----
        repo_org (str): The organization name of the repository.
        repo_name (str): The name of the repository.

    Returns:
    --------
        pd.DataFrame: Data frame containing pull request message data with pull request IDs, timestamps, 
                      and contributor IDs associated with each pull request message thread.
    """
    prm_query = salc.sql.text(f"""
                  SET SCHEMA 'augur_data';
                  SELECT
                      pr.pull_request_id,
                      m.cntrb_id,
                      pr.pr_created_at as timestamp
                  FROM
                      repo_groups rg,
                      repo r,
                      pull_requests pr,
                      pull_request_message_ref prm,
                      message m
                  WHERE
                      rg.repo_group_id = r.repo_group_id AND
                      pr.repo_id = r.repo_id AND
                      pr.pull_request_id = prm.pull_request_id AND
                      m.msg_id = prm.msg_id AND
                      r.repo_git = \'{f"https://github.com/{repo_org}/{repo_name}"}\'
                  ORDER BY
                      timestamp DESC
          """)

    prm_data = pd.read_sql(prm_query, con=engine)
    
    print(prm_data)
    
    # Convert the 'timestamp' column to datetime, this will help in identifying and handling missing values.
    prm_data['timestamp'] = pd.to_datetime(prm_data['timestamp'], errors='coerce')

    # Drop rows where 'timestamp' is NaT (missing)
    prm_data = prm_data.dropna(subset=['timestamp'])

    # reformat pull request message data, combine contributor ids for each pr thread
    prm_data = prm_data.groupby('pull_request_id').agg({'cntrb_id': list, 'timestamp': 'last'}).reset_index()
    # remove pr threads with only one contributor (no connection to be made)
    prm_data = prm_data[prm_data['cntrb_id'].apply(lambda x: len(x) > 1)]
    # prm_data = prm_data.sort_values('timestamp', ascending=False)

    return prm_data

In [10]:
issue_msg_query('tensorflow', 'tensorflow')

,issue_id,cntrb_id,timestamp
0,36881,"[010005d9-9b00-0000-0000-000000000000, 010005d...",2024-07-01 09:39:11
3,36887,"[010005c0-0a00-0000-0000-000000000000, 0105762...",2024-06-17 11:50:42
4,36889,"[010181c9-2600-0000-0000-000000000000, 0100135...",2024-06-14 14:13:49
5,36890,"[01048c28-fe00-0000-0000-000000000000, 01000df...",2024-06-03 11:27:20
7,36893,"[01062378-3f00-0000-0000-000000000000, 01000df...",2024-05-31 16:27:44
...,...,...,...
2759,40043,"[01001ebd-0400-0000-0000-000000000000, 01001eb...",2012-07-20 23:11:39
2763,40049,"[01001ebd-0400-0000-0000-000000000000, 01001eb...",2012-07-20 23:06:02
2768,40054,"[01001ebd-0400-0000-0000-000000000000, 01001eb...",2012-07-20 23:05:55
2771,40057,"[01001ebd-0400-0000-0000-000000000000, 01001eb...",2012-07-20 23:05:52


In [7]:
pr_msg_query('tensorflow', 'tensorflow')

Empty DataFrame
Columns: [pull_request_id, cntrb_id, timestamp]
Index: []


""


In [8]:
commit_query('tensorflow', 'tensorflow')

,cmt_commit_hash,timestamp,author_id,committer_id
0,15ec568b5505727c940b651aeb2a9643b504086c,2024-07-02 00:28:26,0101e3c3-2600-0000-0000-000000000000,010105b7-9400-0000-0000-000000000000
1,125f8bc2f69e62590a633eebcc2dc894e6f6b1ed,2024-07-02 00:11:19,010011eb-6a00-0000-0000-000000000000,010105b7-9400-0000-0000-000000000000
2,2d0547de1279684877e150b7da999fa159b3fa36,2024-07-01 23:08:51,0101e3c3-2600-0000-0000-000000000000,010105b7-9400-0000-0000-000000000000
3,ed31a963c8621a4abdff5b88db877eb726e51884,2024-07-01 22:17:30,010011eb-6a00-0000-0000-000000000000,010105b7-9400-0000-0000-000000000000
4,7c5d07d71a1b1e2616218115ccb3f4ba1b8bbfff,2024-07-01 22:10:09,0102c528-8000-0000-0000-000000000000,010105b7-9400-0000-0000-000000000000
...,...,...,...,...
95148,f2eae4b3d27a4dc6d1f591f55a50fb3e1984a287,2015-12-17 01:54:29,010002ee-8e00-0000-0000-000000000000,01000713-7900-0000-0000-000000000000
95151,881dc225ecb32064681c7bf2229d796565ad7956,2015-12-16 01:34:21,010002ee-8e00-0000-0000-000000000000,01000713-7900-0000-0000-000000000000
95153,1c1ff38d47281570444e2a2b75d7ed945d60e61e,2015-12-15 22:48:46,01002302-0700-0000-0000-000000000000,01000713-7900-0000-0000-000000000000
95162,1b0b52430b4038a90c185bb02b687e76133d259c,2015-12-15 22:46:48,010002ee-8e00-0000-0000-000000000000,01000713-7900-0000-0000-000000000000


In [9]:
pr_query('tensorflow', 'tensorflow')

,pull_request_id,cntrb_id,reviewer,timestamp
